In [0]:
DEBUG = True
DESIGN_PATH = '/datascope/subaru/data/targeting/dSph/draco/netflow/TEST/draco_6_1_020/pfsDesign-0x51600cefa9e6662c.fits'
OBS_TIME = '2025-03-25 14:00:00' # UTC

# Verify bright stars

Load a design file and verify if any of the fiber tips are close to a bright GAIA star

In [0]:
import os, sys
from datetime import datetime
import dateutil.parser
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from matplotlib.gridspec import GridSpec
from scipy.spatial import distance_matrix

In [0]:
plt.rc('font', size=6)

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
if DEBUG and 'debug' not in globals():
    import debugpy
    debugpy.listen(('0.0.0.0', 5698))
    debug = True

In [0]:
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_body
import astropy.units as u

from pfs.utils.coordinates import Subaru_POPT2_PFS
from pfs.datamodel import PfsDesign, FiberStatus, TargetType

from pfs.ga.targeting.targets.dsph import GALAXIES as DSPH_FIELDS
from pfs.ga.targeting.targets.m31 import M31_FIELDS
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, FOV, FP, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color
from pfs.ga.targeting.projection import Pointing, WcsProjection
from pfs.ga.targeting.config.netflow import NetflowConfig
from pfs.ga.targeting.io import GaiaReader

In [0]:
# Load the design file
dir = os.path.dirname(DESIGN_PATH)
fn, ext = os.path.splitext(os.path.basename(DESIGN_PATH))
pfsDesignId = int(fn[10:], 16)
print(fn, hex(pfsDesignId))

design = PfsDesign.read(pfsDesignId, dirName=dir)
design.fiberId.shape

In [0]:
# Query the GAIA database for bright stars
ra, dec = design.raBoresight, design.decBoresight
r = GaiaReader(
    limits = { 'gaia_g': [ -10, 16 ] }
)

gaia = r.cone_search((ra, dec), 50)
gaia.shape

In [0]:
# Histogram of bright star magnitudes

m, _ = gaia.get_magnitude(gaia.photometry['gaia'].magnitudes['g'])
plt.hist(m)

In [0]:
pfi = SubaruPFI(instrument_options={'layout': 'calibration'})

In [0]:
gaia.data.columns

In [0]:
# Conver GAIA star positions into focal plane positions
pointing = Pointing(design.raBoresight, design.decBoresight, posang=design.posAng,
                    obs_time=OBS_TIME)

parallax = np.array(gaia.data['parallax'])
parallax[parallax < 1e-7] = 1e-7

gaia_fp_pos = pfi.radec_to_fp_pos(
    *gaia.get_coords(),
    pointing=pointing,
    epoch=2016.0,
    pmra=np.array(gaia.data['pmra']),
    pmdec=np.array(gaia.data['pmdec']),
    parallax=np.array(gaia.data['parallax']),
    rv=np.array(gaia.data['rv'])
)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=240)

ms = 6

# Broken fibers
mask = (design.fiberStatus == FiberStatus.BROKENFIBER)
ax.scatter(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], marker='x', s=ms, label='broken',
        color='black', edgecolors='none', linewidth=0.5)

# Draw circle around broken cobras with working fibers
mask = (design.fiberStatus == FiberStatus.BROKENCOBRA) & (design.fiberStatus != FiberStatus.BROKENFIBER)
ax.scatter(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], marker='o', s=16, label='broken',
        color='none', edgecolors='k', linewidth=0.5)

# Unassigned fibers
mask = (design.fiberStatus != FiberStatus.BROKENFIBER) & (design.targetType == TargetType.UNASSIGNED)
ax.scatter(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], marker='o', s=ms, label='unassigned',
        color='none', edgecolors='black', linewidth=0.5)

# Good science targets
mask = (design.fiberStatus != FiberStatus.BROKENFIBER) & (design.targetType == TargetType.SCIENCE)
ax.scatter(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], marker='o', s=ms, label='science',
        color='green', edgecolors='none')

# Good sky fibers
mask = (design.fiberStatus != FiberStatus.BROKENFIBER) & (design.targetType == TargetType.SKY)
ax.scatter(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], marker='o', s=ms, label='sky',
        color='cyan', edgecolors='none')

# Good flux standards
mask = (design.fiberStatus != FiberStatus.BROKENFIBER) & (design.targetType == TargetType.FLUXSTD)
ax.scatter(design.pfiNominal[mask][..., 0], design.pfiNominal[mask][..., 1], marker='o', s=ms, label='fluxstd',
        color='red', edgecolors='none')

# Cobra centers
ax.scatter(pfi.bench.cobras.centers.real, pfi.bench.cobras.centers.imag, 
            s=0.2, color='black', edgecolors='none')


# Bright stars
ax.scatter(gaia_fp_pos.real, gaia_fp_pos.imag, marker='o', s=10, label='bright',
        color='none', edgecolors='red')



ax.set_title(
        f'designId=0x{pfsDesignId:016x} ({design.raBoresight:0.02f}, {design.decBoresight:0.02f}, PA={design.posAng:0.01f})\n'
        f'{design.designName}')

ax.set_aspect('equal', adjustable='datalim')

# Make legend horizontal
ax.legend(loc='upper right', ncol=2, fontsize='small')

f.tight_layout()


# Distribution of the distance of closest bright star

In [0]:
# Ignore fibers without cobras
mask = pfi.fiber_map.cobraId[design.fiberId - 1] != 65535

d = distance_matrix(design.pfiNominal[mask], np.stack([gaia_fp_pos.real, gaia_fp_pos.imag], axis=-1))
d.shape

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.5, 2), dpi=240)

bins = np.linspace(0, 10, 20)
hist, bins = np.histogram(d.ravel(), bins=bins)

ax.step(0.5 * (bins[1:] + bins[:-1]), hist, where='mid')

ax.set_xlabel('distance to bright star [mm]')
ax.set_ylabel('fiber count')